In [1]:
import awkward as ak
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch, h5py

from torch.utils.data import DataLoader
from glob import glob
#from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from models.particle_transformer import ParticleTransformer
from utils.data import Dataset, cycle
from tqdm import tqdm

In [11]:
with h5py.File('data/train/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_16.hdf5', 'r') as file:
    X = file['PF_features'][:]

In [ ]:

cut_X.shape

(144855, 4500)

In [ ]:
cut_X = X[:, 5, :]
cut_X.shape

(144855, 4500)

In [38]:
for idx in np.nonzero(np.isinf(cut_X)):
    print(idx)

[19814 28917]
[ 511 1081]


In [41]:
cut_X[19814-2:19814+2, 511-2:511+2]

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.60546875],
       [1.88085938, 0.        ,        inf, 0.        ],
       [8.1953125 , 0.        , 0.        , 0.        ]])

In [10]:
train_files = glob('data/train/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/*.hdf5')
for f in train_files:
    with h5py.File(f, 'r') as file:
        X = file['PF_features'][:, 5, :]
        if not np.isfinite(X).all():
            print(f)
            print(X)

data/train/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/NanoV9MCPostVFP_16.hdf5
[[-1.89971924e-03  0.00000000e+00 -5.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.30789185e-03 -2.32124329e-03  1.03607178e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.90887451e-02 -5.09262085e-03  5.98907471e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  8.04901123e-04 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-9.54627991e-04  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


KeyboardInterrupt: 

In [2]:
train_files = glob('data/train/DYJetsToMuMu_H2ErratumFix_TuneCP5_13TeV-powhegMiNNLO-pythia8-photos/*.hdf5')
dataset_train = Dataset(files=train_files, max_PF_num=1024)
dataloader_train = DataLoader(
    dataset=dataset_train, batch_size=256, sampler=None, 
    num_workers=8, pin_memory=True, drop_last=True, shuffle=True,
)
train_loader = tqdm(dataloader_train, desc=f"Epoch 0")
for index, (X, Y) in enumerate(train_loader):
    X, Y = X.to('cuda'), Y.to('cuda')
    if not torch.isfinite(X).all():
        print(index, torch.isfinite(X).all())
        print(X)
        break

=> Lazy loading 1619244 events from 18 files


Epoch 0:   5%|▍         | 285/6325 [22:43<5:44:02,  3.42s/it]

286 tensor(False, device='cuda:0')
tensor([[[ 4.8656e+01,  8.6875e+00,  7.7422e+00,  ...,  9.0759e-02,
           9.0454e-02,  8.9905e-02],
         [-2.2290e+00, -2.4170e-01,  1.4463e+00,  ..., -4.6729e+00,
          -3.9458e+00,  4.2812e+00],
         [-3.5645e-02, -1.9509e+00,  9.8251e-02,  ...,  9.5886e-01,
           1.3101e+00, -1.8340e+00],
         ...,
         [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 4.8625e+01, -3.2234e+00,  7.7048e+00,  ...,  5.2137e-02,
           2.3318e-02, -2.3390e-02],
         [-1.7340e+00, -8.0674e+00,  7.5946e-01,  ...,  7.4290e-02,
           8.7397e-02, -8.6809e-02]],

        [[ 4.5844e+01,  4.1250e+01,  5.4414e+00,  ...,  1.3000e-01,
           1.2976e-01,  1.2854e-01],
         [-1.6799e+00, -1.5198e+00,  2.3740e+00,  ..., -4.8096e+00,
           3.9346e+00, -5.0479e+00],
         [-1.5925e+00,  1.6663e+00, -2.9985e+00,  ..., -2.7861e+00,
          -1.1379e+00, -3.1304e+00],
    

Epoch 0:   5%|▍         | 286/6325 [23:18<8:12:13,  4.89s/it]


In [ ]:
def _check_finite(t, name):
    if not torch.isfinite(t).all():
        raise RuntimeError(f"[Non-finite detected] {name} contains NaN/Inf")


def quick_data_sanity(loader, device, n_batches=100):
    cnt = 0
    for X, Y in loader:
        X, Y = X.to(device), Y.to(device)
        if not (torch.isfinite(X).all() and torch.isfinite(Y).all()):
            raise RuntimeError("Dataset contains NaN/Inf in X or Y")
            return X, Y


In [ ]:
for f in glob('/data/fudawei/GenMET/data/test/*/*.hdf5'):
    with h5py.File(f, 'r') as file:
        arr = file['event_truths'][:]
        print(f, np.max(arr), np.min(arr))
    

In [ ]:
dataset = Dataset(glob('/data/fudawei/GenMET/data/train/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/*'), max_PF_num=1000)
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True, drop_last=False)


In [ ]:
lazy_files=[h5py.File(f, 'r') for f in glob('/data/fudawei/GenMET/data/train/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/*')]
lazy_files[0].attrs['n_events']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(torch.abs(dataset.targets[torch.any(torch.abs(dataset.targets)>1e3, dim=-1)]).flatten(), range=(1e3, 1e4))

In [ ]:
dataset.features[torch.any(torch.abs(dataset.targets)>1e3, dim=-1)]

In [ ]:
dataset.targets[torch.any(dataset.targets == torch.min(dataset.targets), dim=-1)]

In [ ]:
with open('/data/fudawei/GenMET/output/MET/b=512_lr=0.001_EMAmu=0.999/model/checkpoint_2050.pt', 'rb') as f:
    loaded_obj = torch.load(f, map_location='cpu')

In [ ]:
loaded_obj['model']['embed.input_bn.running_var']

In [ ]:
for k, v in loaded_obj['model'].items():
    print(k)
    print(v)

In [ ]:
dataloader = cycle(DataLoader(dataset=dataset, batch_size=1024, shuffle=True, drop_last=False))
inputs, targets = next(dataloader)

In [ ]:
inputs.shape

In [ ]:
model = ParticleTransformer(
    input_dim=len(features),
    output_dim=len(targets),  
    # network configurations
    pair_input_dim=4,
    use_pre_activation_pair=False,
    embed_dims=[128, 512, 128],
    pair_embed_dims=[64, 64, 64],
    num_heads=8,
    num_layers=8,
    num_cls_layers=2,
    block_params=None,
    cls_block_params={'dropout': 0, 'attn_dropout': 0, 'activation_dropout': 0},
    fc_params=[],
    activation='gelu',
    # misc
    trim=True,
    for_inference=False,
)

In [ ]:
outputs = model(inputs)

In [ ]:
import torch